# Project: Channel Video Uploads Sorted by Length

In [130]:
## Only needs to be done once, then comment it out after.
# !pip install --upgrade google-api-python-client
# !pip install isodate

from googleapiclient.discovery import build
import pandas as pd
from IPython.display import display
import isodate

In [131]:
api_key = ''
channel_id = "UC8butISFwT-Wl7EV0hUK0BQ"
playlist_id = "UU8butISFwT-Wl7EV0hUK0BQ"

# Provided by YouTube v3 Channel list documentation.

api_service_name = "youtube"
api_version = "v3"

# Create an API client
youtube = build(api_service_name, api_version, developerKey = api_key)

In [132]:
search_term = "backend"

# Test run data for a couple videos

In [ ]:
request = youtube.search().list(
        part = "snippet",
        channelId = "UC8butISFwT-Wl7EV0hUK0BQ",
        maxResults = 50,
        type = "video",
        q = "backend"
)

response = request.execute()

display(response)

# Get all the video ids

In [ ]:
def get_video_ids():
    
    # List to hold our video ids.
    video_ids = []

    request = youtube.search().list(
        part = "snippet",
        channelId = channel_id,
        maxResults = 50,
        type = "video",
        q = search_term
    )

    response = request.execute()

    # Populate our video ids list.
    for item in response['items']:
        video_ids.append(item['id']['videoId'])
       
    # Since we want all the videos, not just one page's worth, we have to
    # iterate over each new page.
    next_page_token = response.get('nextPageToken')
    
    while next_page_token is not None:
        request = youtube.search().list(
            part = "snippet",
            channelId = channel_id,
            maxResults = 50,
            type = "video",
            q = search_term,
            pageToken = next_page_token
        )

        response = request.execute()
        
        for item in response['items']:
            video_ids.append(item['id']['videoId'])
            
        next_page_token = response.get('nextPageToken')
    
    return video_ids

In [ ]:
video_ids = get_video_ids()

len(video_ids)

# Get details for all the video ids

In [ ]:
def get_video_details(youtube, video_ids):
    
    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part = "snippet,contentDetails",
            id = ",".join(video_ids[i : i + 50])
        )
        
        response = request.execute()
        
        for video in response['items']:
            stats_to_keep = {'snippet': ['title', 'description', 'publishedAt'],
                             'contentDetails': ['duration']
                            }
            video_info = {}
            video_info['video_id'] = video['id']
            
            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None
            
            all_video_info.append(video_info)
            
    return pd.DataFrame(all_video_info)

In [ ]:
video_df = get_video_details(youtube, video_ids)

video_df

In [ ]:
# Convert duration column to standard hours, minutes, seconds format.

video_df['duration_time'] = video_df['duration'].apply(lambda x: isodate.parse_duration(x))
video_df['duration_seconds'] = video_df['duration'].apply(lambda x: isodate.parse_duration(x).total_seconds())
video_df['duration_time'] = video_df['duration_time'].astype('timedelta64[s]')

video_df

In [139]:
# Sort the videos by the durationTime column to find the shortest videos.

video_df.sort_values(by = 'duration_time', ascending = True).head(50)

,video_id,title,description,publishedAt,duration,duration_time,duration_seconds
53,SIj4pzF0uk8,A Day in the Life of freeCodeCamp U.S. Staff (...,This was not a normal day. :)\n\nMusic:\n8-Bit...,2023-03-07T14:37:39Z,PT19S,0 days 00:00:19,19.0
23,YY2ZhUQZyE4,Why Node and React?,,2023-03-01T15:53:31Z,PT31S,0 days 00:00:31,31.0
433,8IOHjh3VQMU,Webhooks Explained with a Roadtrip #shorts,Watch the full course: https://www.youtube.com...,2023-01-24T17:11:22Z,PT59S,0 days 00:00:59,59.0
202,j3YM5eUjw0E,100 Days of Code,#code #developer #programming #dev #100daysofc...,2024-02-14T19:17:27Z,PT59S,0 days 00:00:59,59.0
301,JQ_BCly-heU,Computer Basics 20: What Do Programmers Do?,http://FreeCodeCamp.com is a community of busy...,2015-07-17T08:31:55Z,PT1M32S,0 days 00:01:32,92.0
255,5ZzioWgcObg,Web Design: Development vs Design,I'm Anthony also known as dapperAuteur on Free...,2016-03-27T18:32:33Z,PT1M50S,0 days 00:01:50,110.0
158,WvXsY736Kz0,What do computer programmers actually do?,What do programmers actually do? What can they...,2018-11-27T15:03:33Z,PT4M23S,0 days 00:04:23,263.0
73,ABlaMXkUwzY,Debugging JavaScript - Are you doing it wrong?,Learn a better way to debug your JavaScript.\n...,2018-11-01T19:45:00Z,PT4M44S,0 days 00:04:44,284.0
329,PfLBugPdgSk,Learn to Code: 5 Secret Weapons,Learning web development is difficult. This is...,2018-07-03T17:54:08Z,PT5M24S,0 days 00:05:24,324.0
318,VRz0nbax0uI,20 String Methods in 7 Minutes - Beau teaches ...,"String methods featured in this video: charAt,...",2017-02-15T18:00:37Z,PT7M,0 days 00:07:00,420.0


In [157]:
# Create a copy of the DataFrame
filtered_video_df = video_df.copy()

# Filter DataFrame to keep rows where duration is greater than or equal to 25 minutes (1500 seconds)
filtered_video_df = filtered_video_df[filtered_video_df['duration_seconds'] >= 1500]

# Drop the 'duration_seconds' column if you no longer need it
filtered_video_df.drop(columns = ['duration_seconds'], inplace = True)

In [158]:
filtered_video_df.sort_values(by = 'duration_time', ascending = True).reset_index().iloc[0:60]

,index,video_id,title,description,publishedAt,duration,duration_time
0,196,A5GvnU1JxdQ,Build Better and Discoverable APIs with GraphQL,This talk is an introduction to GraphQL with d...,2018-09-25T13:58:38Z,PT25M17S,0 days 00:25:17
1,269,1BT4efG-ItA,Introduction to Front End Testing,Get started with testing for the Front End. Le...,2018-11-01T15:00:00Z,PT25M32S,0 days 00:25:32
2,256,6iZAl0jCHGc,Everyone is a Developer - Talk by Jessica Lord,"Development isn't just for professionals, it's...",2018-12-04T20:11:15Z,PT27M57S,0 days 00:27:57
3,254,mZdznKc5ZaM,One weird trick to becoming a better software ...,Those of us who have worked in software develo...,2018-06-05T14:05:06Z,PT28M22S,0 days 00:28:22
4,230,NSts93C9UeE,JAMstack Tutorial - Full site using Netlify & ...,In this tutorial you will learn to build a cli...,2017-09-22T16:47:42Z,PT29M51S,0 days 00:29:51
5,182,vXJpOHz3_sY,The All Powerful Front End Developer - Chris C...,"The internet is, without metaphor, just a bunc...",2018-11-30T14:26:36Z,PT31M13S,0 days 00:31:13
6,419,fsCjFHuMXj0,Build a Node.js API - tutorial,"Create basic CRUD routes with Node.js, Express...",2017-12-05T17:00:20Z,PT33M10S,0 days 00:33:10
7,374,fsCjFHuMXj0,Build a Node.js API - tutorial,"Create basic CRUD routes with Node.js, Express...",2017-12-05T17:00:20Z,PT33M10S,0 days 00:33:10
8,240,MMg4n68wodo,"All about GRAND Stack: GraphQL, React, Apollo,...","In this presentation, we explore application d...",2018-06-07T15:02:33Z,PT33M30S,0 days 00:33:30
9,300,xPEMup5SPTM,TypeScript 101 (tutorial),Introduction to TypeScript. TypeScript is an o...,2018-05-18T16:30:04Z,PT33M59S,0 days 00:33:59


In [140]:
len(filtered_video_df)

400